In [ ]:
#!pip install pyspark==3.3.2

In [1]:
#restart kernel to suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.3.2
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 17.0.3
Branch HEAD
Compiled by user liangchi on 2023-02-10T19:57:40Z
Revision 5103e00c4ce5fcc4264ca9c4df12295d42557af6
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType
from pyspark.sql.functions import col


In [ ]:
!wget "https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q1_2022.zip"


In [ ]:
!unzip '/content/data_Q1_2022.zip'

In [5]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size", "8g")\
        .config("spark.driver.memory", "8g")\
        .config("spark.driver.maxResultSize", 0)\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/23 11:44:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
%%time
df = spark.read.csv("data_Q1_2022", header=True, inferSchema=True)

CPU times: user 70.9 ms, sys: 27.5 ms, total: 98.4 ms
Wall time: 2min 16s


In [10]:
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: long (nullable = true)
 |-- failure: integer (nullable = true)
 |-- smart_1_normalized: integer (nullable = true)
 |-- smart_1_raw: integer (nullable = true)
 |-- smart_2_normalized: integer (nullable = true)
 |-- smart_2_raw: integer (nullable = true)
 |-- smart_3_normalized: integer (nullable = true)
 |-- smart_3_raw: integer (nullable = true)
 |-- smart_4_normalized: integer (nullable = true)
 |-- smart_4_raw: integer (nullable = true)
 |-- smart_5_normalized: integer (nullable = true)
 |-- smart_5_raw: integer (nullable = true)
 |-- smart_7_normalized: integer (nullable = true)
 |-- smart_7_raw: long (nullable = true)
 |-- smart_8_normalized: integer (nullable = true)
 |-- smart_8_raw: integer (nullable = true)
 |-- smart_9_normalized: integer (nullable = true)
 |-- smart_9_raw: integer (nullable = true)
 |-- smart_10_normalized: integer

In [11]:
%%time
#Selecting all raw values
columns = ["serial_number"]+["model"]+["capacity_bytes"]+["failure"]+[col_name for col_name in df.columns if 'raw' in col_name]
df = df.select(columns)
df.printSchema()

root
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: long (nullable = true)
 |-- failure: integer (nullable = true)
 |-- smart_1_raw: integer (nullable = true)
 |-- smart_2_raw: integer (nullable = true)
 |-- smart_3_raw: integer (nullable = true)
 |-- smart_4_raw: integer (nullable = true)
 |-- smart_5_raw: integer (nullable = true)
 |-- smart_7_raw: long (nullable = true)
 |-- smart_8_raw: integer (nullable = true)
 |-- smart_9_raw: integer (nullable = true)
 |-- smart_10_raw: integer (nullable = true)
 |-- smart_11_raw: integer (nullable = true)
 |-- smart_12_raw: integer (nullable = true)
 |-- smart_13_raw: integer (nullable = true)
 |-- smart_15_raw: string (nullable = true)
 |-- smart_16_raw: integer (nullable = true)
 |-- smart_17_raw: integer (nullable = true)
 |-- smart_18_raw: integer (nullable = true)
 |-- smart_22_raw: integer (nullable = true)
 |-- smart_23_raw: integer (nullable = true)
 |-- smart_24_raw: integer (nul

In [12]:
#Removing columns with string values
cols = ("smart_255_raw","smart_250_raw", "smart_252_raw", "smart_251_raw", "smart_250_raw", "smart_178_raw", "smart_176_raw", "smart_175_raw", "smart_164_raw", "smart_163_raw", "smart_161_raw", "smart_160_raw", "smart_15_raw")
df = df.drop(*cols)
df.printSchema()

root
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: long (nullable = true)
 |-- failure: integer (nullable = true)
 |-- smart_1_raw: integer (nullable = true)
 |-- smart_2_raw: integer (nullable = true)
 |-- smart_3_raw: integer (nullable = true)
 |-- smart_4_raw: integer (nullable = true)
 |-- smart_5_raw: integer (nullable = true)
 |-- smart_7_raw: long (nullable = true)
 |-- smart_8_raw: integer (nullable = true)
 |-- smart_9_raw: integer (nullable = true)
 |-- smart_10_raw: integer (nullable = true)
 |-- smart_11_raw: integer (nullable = true)
 |-- smart_12_raw: integer (nullable = true)
 |-- smart_13_raw: integer (nullable = true)
 |-- smart_16_raw: integer (nullable = true)
 |-- smart_17_raw: integer (nullable = true)
 |-- smart_18_raw: integer (nullable = true)
 |-- smart_22_raw: integer (nullable = true)
 |-- smart_23_raw: integer (nullable = true)
 |-- smart_24_raw: integer (nullable = true)
 |-- smart_165_raw: long (null

In [9]:
df.show()

23/03/22 17:09:06 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------+--------------------+--------------+-------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--

In [9]:
df.na.fill(value=0)

DataFrame[serial_number: string, model: string, capacity_bytes: bigint, failure: int, smart_1_raw: int, smart_2_raw: int, smart_3_raw: int, smart_4_raw: int, smart_5_raw: int, smart_7_raw: bigint, smart_8_raw: int, smart_9_raw: int, smart_10_raw: int, smart_11_raw: int, smart_12_raw: int, smart_13_raw: int, smart_15_raw: string, smart_16_raw: int, smart_17_raw: int, smart_18_raw: int, smart_22_raw: int, smart_23_raw: int, smart_24_raw: int, smart_160_raw: string, smart_161_raw: string, smart_163_raw: string, smart_164_raw: string, smart_165_raw: bigint, smart_166_raw: int, smart_167_raw: int, smart_168_raw: int, smart_169_raw: int, smart_170_raw: bigint, smart_171_raw: int, smart_172_raw: int, smart_173_raw: bigint, smart_174_raw: int, smart_175_raw: string, smart_176_raw: string, smart_177_raw: int, smart_178_raw: string, smart_179_raw: int, smart_180_raw: int, smart_181_raw: int, smart_182_raw: int, smart_183_raw: int, smart_184_raw: int, smart_187_raw: int, smart_188_raw: bigint, sm

In [9]:
df.createOrReplaceTempView('my_table')

In [10]:
%%time
df.count()

CPU times: user 29.5 ms, sys: 12.5 ms, total: 42 ms
Wall time: 12.2 s


18845260

In [11]:
most_failure = spark.sql("""SELECT model, COUNT(*) AS num_failures
FROM my_table
WHERE failure = 1
GROUP BY model
ORDER BY num_failures DESC
LIMIT 10;"""
).cache()

In [12]:
%%time
most_failure.show()

+--------------------+------------+
|               model|num_failures|
+--------------------+------------+
|         ST4000DM000|         117|
| TOSHIBA MG07ACA14TA|          88|
|        ST8000NM0055|          86|
|       ST12000NM0008|          73|
|         ST8000DM002|          38|
|       ST14000NM001G|          24|
|       ST16000NM001G|          24|
|HGST HMS5C4040ALE640|          21|
|HGST HUH721212ALE604|          21|
|       ST12000NM001G|          20|
+--------------------+------------+

CPU times: user 57.4 ms, sys: 22.5 ms, total: 79.8 ms
Wall time: 1min 1s


In [15]:
#using sql as it is much faster
zero = spark.sql("SELECT COUNT(*) FROM my_table WHERE failure = 0").collect()[0][0]
one = spark.sql("SELECT COUNT(*) FROM my_table WHERE failure = 1").collect()[0][0]
print(zero,one)

18844596 664


In [21]:
total = zero + one
print("Total number of samples: ", total)
print("Positive samples (Failed drives): ", one, "\npercentage of total: {:.4f}%".format(one*100/total))

Total number of samples:  18845260
Positive samples (Failed drives):  664 
percentage of total: 0.0035%


In [13]:
# filter the DataFrame to only include rows where the "model" column is equal to "ST4000DM000"
df_filtered = df.filter(col("model") == "ST4000DM000")

# show the first 5 rows of the filtered DataFrame
df_filtered.show(5)

23/03/23 11:49:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+-------------+-----------+--------------+-------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------------

In [14]:
df = df_filtered
df.show(5)

+-------------+-----------+--------------+-------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------

In [24]:
df.createOrReplaceTempView('my_table')

In [25]:
zero = spark.sql("SELECT COUNT(*) FROM my_table WHERE failure = 0").collect()[0][0]
one = spark.sql("SELECT COUNT(*) FROM my_table WHERE failure = 1").collect()[0][0]
print(zero,one)

1670278 117


In [26]:
total = zero + one
print("Total number of samples: ", total)
print("Positive samples (Failed drives): ", one, "\npercentage of total: {:.4f}%".format(one*100/total))

Total number of samples:  1670395
Positive samples (Failed drives):  117 
percentage of total: 0.0070%


In [15]:
#Selections 5 Smart features
columns = ["failure"]+["smart_5_raw", "smart_187_raw", "smart_188_raw", "smart_197_raw", "smart_198_raw"]
df_b = df.select(columns)
df_b.printSchema()

root
 |-- failure: integer (nullable = true)
 |-- smart_5_raw: integer (nullable = true)
 |-- smart_187_raw: integer (nullable = true)
 |-- smart_188_raw: long (nullable = true)
 |-- smart_197_raw: integer (nullable = true)
 |-- smart_198_raw: integer (nullable = true)



In [16]:
#!pip install pyarrow
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [17]:
%%time
pandas_df = df.toPandas()
pandas_df_b = df_b.toPandas()

CPU times: user 560 ms, sys: 880 ms, total: 1.44 s
Wall time: 2min 2s


In [18]:
pandas_df.head()

,serial_number,model,capacity_bytes,failure,smart_1_raw,smart_2_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,...,smart_235_raw,smart_240_raw,smart_241_raw,smart_242_raw,smart_244_raw,smart_245_raw,smart_246_raw,smart_247_raw,smart_248_raw,smart_254_raw
0,Z305DEMX,ST4000DM000,4000787030016,0,14386032.0,NaN,0.0,16.0,0.0,312626083.0,...,NaN,54885.0,6.522921e+10,3.345606e+11,NaN,NaN,NaN,NaN,NaN,NaN
1,S301NGZV,ST4000DM000,4000787030016,0,37921224.0,NaN,0.0,19.0,0.0,339757984.0,...,NaN,56133.0,6.690510e+10,1.063485e+12,NaN,NaN,NaN,NaN,NaN,NaN
2,S3010M7R,ST4000DM000,4000787030016,0,240150664.0,NaN,0.0,19.0,0.0,651676751.0,...,NaN,60024.0,7.388683e+10,4.323190e+11,NaN,NaN,NaN,NaN,NaN,NaN
3,S3010M7H,ST4000DM000,4000787030016,0,172425960.0,NaN,0.0,26.0,0.0,303794012.0,...,NaN,59081.0,6.617698e+10,3.839506e+11,NaN,NaN,NaN,NaN,NaN,NaN
4,S3010M7E,ST4000DM000,4000787030016,0,41968136.0,NaN,0.0,19.0,0.0,798267093.0,...,NaN,60006.0,7.435509e+10,4.196393e+11,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
pandas_df_b.head()

,failure,smart_5_raw,smart_187_raw,smart_188_raw,smart_197_raw,smart_198_raw
0,0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,1.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0


In [20]:
pandas_df.dtypes

serial_number      object
model              object
capacity_bytes      int64
failure             int32
smart_1_raw       float64
                   ...   
smart_245_raw     float64
smart_246_raw     float64
smart_247_raw     float64
smart_248_raw     float64
smart_254_raw     float64
Length: 79, dtype: object

In [21]:
pandas_df = pandas_df.drop(["serial_number", "model", "capacity_bytes"], axis=1)
pandas_df.head()

,failure,smart_1_raw,smart_2_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_8_raw,smart_9_raw,smart_10_raw,...,smart_235_raw,smart_240_raw,smart_241_raw,smart_242_raw,smart_244_raw,smart_245_raw,smart_246_raw,smart_247_raw,smart_248_raw,smart_254_raw
0,0,14386032.0,NaN,0.0,16.0,0.0,312626083.0,NaN,55292.0,0.0,...,NaN,54885.0,6.522921e+10,3.345606e+11,NaN,NaN,NaN,NaN,NaN,NaN
1,0,37921224.0,NaN,0.0,19.0,0.0,339757984.0,NaN,56267.0,0.0,...,NaN,56133.0,6.690510e+10,1.063485e+12,NaN,NaN,NaN,NaN,NaN,NaN
2,0,240150664.0,NaN,0.0,19.0,0.0,651676751.0,NaN,60237.0,0.0,...,NaN,60024.0,7.388683e+10,4.323190e+11,NaN,NaN,NaN,NaN,NaN,NaN
3,0,172425960.0,NaN,0.0,26.0,0.0,303794012.0,NaN,59929.0,0.0,...,NaN,59081.0,6.617698e+10,3.839506e+11,NaN,NaN,NaN,NaN,NaN,NaN
4,0,41968136.0,NaN,0.0,19.0,0.0,798267093.0,NaN,60243.0,0.0,...,NaN,60006.0,7.435509e+10,4.196393e+11,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
pandas_df = pandas_df.fillna(0)
pandas_df.head()

,failure,smart_1_raw,smart_2_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_8_raw,smart_9_raw,smart_10_raw,...,smart_235_raw,smart_240_raw,smart_241_raw,smart_242_raw,smart_244_raw,smart_245_raw,smart_246_raw,smart_247_raw,smart_248_raw,smart_254_raw
0,0,14386032.0,0.0,0.0,16.0,0.0,312626083.0,0.0,55292.0,0.0,...,0.0,54885.0,6.522921e+10,3.345606e+11,0.0,0.0,0.0,0.0,0.0,0.0
1,0,37921224.0,0.0,0.0,19.0,0.0,339757984.0,0.0,56267.0,0.0,...,0.0,56133.0,6.690510e+10,1.063485e+12,0.0,0.0,0.0,0.0,0.0,0.0
2,0,240150664.0,0.0,0.0,19.0,0.0,651676751.0,0.0,60237.0,0.0,...,0.0,60024.0,7.388683e+10,4.323190e+11,0.0,0.0,0.0,0.0,0.0,0.0
3,0,172425960.0,0.0,0.0,26.0,0.0,303794012.0,0.0,59929.0,0.0,...,0.0,59081.0,6.617698e+10,3.839506e+11,0.0,0.0,0.0,0.0,0.0,0.0
4,0,41968136.0,0.0,0.0,19.0,0.0,798267093.0,0.0,60243.0,0.0,...,0.0,60006.0,7.435509e+10,4.196393e+11,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
pandas_df_b = pandas_df_b.fillna(0)
pandas_df_b.head()

,failure,smart_5_raw,smart_187_raw,smart_188_raw,smart_197_raw,smart_198_raw
0,0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,1.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0


In [27]:
X = pandas_df.drop('failure', axis=1).values
y = pandas_df['failure'].values
X_b = pandas_df_b.drop('failure', axis=1).values
y_b = pandas_df_b['failure'].values

In [28]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MaxAbsScaler
from sklearn.utils.parallel import delayed
from sklearn.metrics import make_scorer
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, f1_score, precision_score, recall_score, accuracy_score, confusion_matrix


def fdr_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    fdr = tp / (tp + fn)
    return fdr

def far_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    far = fp / (fp + tn)
    return far
    


scoring = {'precision': 'precision','gmean': make_scorer(geometric_mean_score),'f1': 'f1' , 'recall':'recall', 'fdr': make_scorer(fdr_score), 'far': make_scorer(far_score)}

In [24]:
%%time

#cv = 5 and all features
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant',fill_value= 0)),
    ('scaler', MaxAbsScaler()),
    ('classifier', BalancedRandomForestClassifier(random_state=42))
])

# evaluate the pipeline using cross-validation with G-mean
BRF_scores = cross_validate(pipeline, X, y, cv=5, scoring=scoring)
# print the mean G-mean score and standard deviation
print("Mean G-mean: {:.3f} (+/- {:.3f})".format(BRF_scores['test_gmean'].mean(), BRF_scores['test_gmean'].std()))
print("F1: ", BRF_scores['test_f1'].mean())
print("Precision: ", BRF_scores['test_precision'].mean())
print("FDR: ", BRF_scores['test_fdr'].mean())
print("FAR :", BRF_scores['test_far'].mean())
print("Recall: ", BRF_scores['test_recall'].mean())

Mean G-mean: 0.887 (+/- 0.040)
F1:  0.002895544189544485
Precision:  0.0014503493212233791
FDR:  0.8217391304347826
FAR : 0.03981971957646102
Recall:  0.8217391304347826
CPU times: user 3min 2s, sys: 14.6 s, total: 3min 16s
Wall time: 3min 18s


In [25]:
%%time
#Pipeline with 5 features
BRF_scores_b = cross_validate(pipeline, X_b, y_b, cv=5, scoring=scoring)

# print the mean G-mean score and standard deviation
print("Mean G-mean: {:.3f} (+/- {:.3f})".format(BRF_scores_b['test_gmean'].mean(), BRF_scores_b['test_gmean'].std()))
print("F1: ", BRF_scores_b['test_f1'].mean())
print("Precision: ", BRF_scores_b['test_precision'].mean())
print("FDR: ", BRF_scores_b['test_fdr'].mean())
print("FAR :", BRF_scores_b['test_far'].mean())
print("Recall: ", BRF_scores_b['test_recall'].mean())

Mean G-mean: 0.907 (+/- 0.032)
F1:  0.0021760452620553394
Precision:  0.0010893908036328937
FDR:  0.8728260869565216
FAR : 0.05609125755276646
Recall:  0.8728260869565216
CPU times: user 2min 40s, sys: 7.79 s, total: 2min 48s
Wall time: 2min 48s


#Random Forest

In [26]:
%%time
#cv = 5 and Random forest and all features
from sklearn.ensemble import RandomForestClassifier
RF_pipeline =  Pipeline([("impute", SimpleImputer(strategy='constant',fill_value= 0)),("scale", MaxAbsScaler()),("RF",RandomForestClassifier(verbose=2, n_jobs=8))])
RF_scores = cross_validate(RF_pipeline, X, y, cv=5, scoring=scoring)
sorted(RF_scores.keys())
# print the mean G-mean score and standard deviation
print("Mean G-mean: {:.3f} (+/- {:.3f})".format(RF_scores['test_gmean'].mean(), RF_scores['test_gmean'].std()))
print("F1: ", RF_scores['test_f1'].mean())
print("Precision: ", RF_scores['test_precision'].mean())
print("FDR: ", RF_scores['test_fdr'].mean())
print("FAR :", RF_scores['test_far'].mean())
print("Recall: ", RF_scores['test_recall'].mean())

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   14.5s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:   47.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   13.7s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:   46.6s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   15.7s
[Parallel(n_jobs=8)]: Done 100

Mean G-mean: 0.000 (+/- 0.000)
F1:  0.0
Precision:  0.0
FDR:  0.0
FAR : 4.789621488946221e-06
Recall:  0.0
CPU times: user 19.1 s, sys: 7.73 s, total: 26.8 s
Wall time: 4min 9s


In [28]:
%%time
#cv = 5 and Random forest and 5 features
from sklearn.ensemble import RandomForestClassifier
RF_b_pipeline =  Pipeline([("impute", SimpleImputer(strategy='constant',fill_value= 0)),("scale", MaxAbsScaler()),("RF",RandomForestClassifier(verbose=2, n_jobs=8))])
RF_scores_b = cross_validate(RF_b_pipeline, X_b, y_b, cv=5, scoring=scoring)

# print the mean G-mean score and standard deviation
print("Mean G-mean: {:.3f} (+/- {:.3f})".format(RF_scores_b['test_gmean'].mean(), RF_scores_b['test_gmean'].std()))
print("F1: ", RF_scores_b['test_f1'].mean())
print("Precision: ", RF_scores_b['test_precision'].mean())
print("FDR: ", RF_scores_b['test_fdr'].mean())
print("FAR :", RF_scores_b['test_far'].mean())
print("Recall: ", RF_scores_b['test_recall'].mean())

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    6.5s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    6.4s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done 100

Mean G-mean: 0.101 (+/- 0.126)
F1:  0.0419047619047619
Precision:  0.10857142857142858
FDR:  0.02608695652173913
FAR : 1.4967552143065255e-05
Recall:  0.02608695652173913
CPU times: user 10.1 s, sys: 1.54 s, total: 11.6 s
Wall time: 37.6 s


In [28]:
%%time
from sklearn.tree import DecisionTreeClassifier
# cv=5 and all features + DT
DT_pipe =  Pipeline([("impute", SimpleImputer(strategy='constant',fill_value= 0)),("scale", MaxAbsScaler()),("DT", DecisionTreeClassifier())])
DT_scores = cross_validate(DT_pipe, X, y, cv=5, scoring=scoring, n_jobs=-1)
# print the mean G-mean score and standard deviation
print("Mean G-mean: {:.3f} (+/- {:.3f})".format(DT_scores['test_gmean'].mean(), DT_scores['test_gmean'].std()))
print("F1: ", DT_scores['test_f1'].mean())
print("Precision: ", DT_scores['test_precision'].mean())
print("FDR: ", DT_scores['test_fdr'].mean())
print("FAR :", DT_scores['test_far'].mean())
print("Recall: ", DT_scores['test_recall'].mean())

Mean G-mean: 0.254 (+/- 0.062)
F1:  0.06834288101205395
Precision:  0.07742426129016809
FDR:  0.06847826086956522
FAR : 0.0001388989317759209
Recall:  0.06847826086956522
CPU times: user 302 ms, sys: 1.12 s, total: 1.42 s
Wall time: 4min 3s


In [29]:
%%time
from sklearn.tree import DecisionTreeClassifier
# cv=5 and 5 features + DT
DT_b_pipe =  Pipeline([("impute", SimpleImputer(strategy='constant',fill_value= 0)),("scale", MaxAbsScaler()),("DT", DecisionTreeClassifier())])
DT_b_scores = cross_validate(DT_b_pipe, X_b, y_b, cv=5, scoring=scoring)
# print the mean G-mean score and standard deviation
print("Mean G-mean: {:.3f} (+/- {:.3f})".format(DT_b_scores['test_gmean'].mean(), DT_scores['test_gmean'].std()))
print("F1: ", DT_b_scores['test_f1'].mean())
print("Precision: ", DT_b_scores['test_precision'].mean())
print("FDR: ", DT_b_scores['test_fdr'].mean())
print("FAR :", DT_b_scores['test_far'].mean())
print("Recall: ", DT_b_scores['test_recall'].mean())

Mean G-mean: 0.258 (+/- 0.062)
F1:  0.07322347166396413
Precision:  0.08653603706235284
FDR:  0.06847826086956521
FAR : 6.525856791975828e-05
Recall:  0.06847826086956521
CPU times: user 4.91 s, sys: 304 ms, total: 5.21 s
Wall time: 5.22 s


In [30]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
#cv = 5 and RFE + Random Forest + 10 selected features and removing 5 features each time
from joblib import parallel_backend


rfecv = RFE(DecisionTreeClassifier(), n_features_to_select=10, step=5)
rfe_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant',fill_value= 0)),
    ('scaler', MaxAbsScaler()),
    ('feature', rfecv),
    ('classifier', DecisionTreeClassifier())
])
rfe_scores = cross_validate(rfe_pipeline, X, y, cv=5, scoring=scoring, n_jobs=7)
# print the mean G-mean score and standard deviation
print("Mean G-mean: {:.3f} (+/- {:.3f})".format(rfe_scores['test_gmean'].mean(), rfe_scores['test_gmean'].std()))
print("F1: ", rfe_scores['test_f1'].mean())
print("Precision: ", rfe_scores['test_precision'].mean())
print("FDR: ", rfe_scores['test_fdr'].mean())
print("FAR :", rfe_scores['test_far'].mean())
print("Recall: ", rfe_scores['test_recall'].mean())

Mean G-mean: 0.284 (+/- 0.068)
F1:  0.06415929131275702
Precision:  0.052534546063957833
FDR:  0.08550724637681159
FAR : 0.0001209379394594968
Recall:  0.08550724637681159
CPU times: user 389 ms, sys: 982 ms, total: 1.37 s
Wall time: 12min 31s


In [31]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
# cv=5 and all features + GBDT
GBDT_pipe =  Pipeline([("impute", SimpleImputer(strategy='constant',fill_value= 0)),("scale", MaxAbsScaler()),("GBDT", GradientBoostingClassifier())])
GBDT_scores = cross_validate(GBDT_pipe, X, y, cv=5, scoring=scoring, n_jobs=-1)
# print the mean G-mean score and standard deviation
print("Mean G-mean: {:.3f} (+/- {:.3f})".format(GBDT_scores['test_gmean'].mean(), GBDT_scores['test_gmean'].std()))
print("F1: ", GBDT_scores['test_f1'].mean())
print("Precision: ", GBDT_scores['test_precision'].mean())
print("FDR: ", GBDT_scores['test_fdr'].mean())
print("FAR :", GBDT_scores['test_far'].mean())
print("Recall: ", GBDT_scores['test_recall'].mean())


Mean G-mean: 0.376 (+/- 0.111)
F1:  0.11039520885140877
Precision:  0.08929137311490251
FDR:  0.15398550724637683
FAR : 0.00012033922848433386
Recall:  0.15398550724637683
CPU times: user 888 ms, sys: 8.39 s, total: 9.28 s
Wall time: 58min 40s


In [33]:
%%time
# cv=5 and 5 features + GBDT
GBDT_b_pipe =  Pipeline([("impute", SimpleImputer(strategy='constant',fill_value= 0)),("scale", MaxAbsScaler()),("GBDT", GradientBoostingClassifier())])
GBDT_b_scores = cross_validate(GBDT_b_pipe, X_b, y_b, cv=5, scoring=scoring, n_jobs=-1)
# print the mean G-mean score and standard deviation
print("Mean G-mean: {:.3f} (+/- {:.3f})".format(GBDT_b_scores['test_gmean'].mean(), GBDT_b_scores['test_gmean'].std()))
print("F1: ", GBDT_b_scores['test_f1'].mean())
print("Precision: ", GBDT_b_scores['test_precision'].mean())
print("FDR: ", GBDT_b_scores['test_fdr'].mean())
print("FAR :", GBDT_b_scores['test_far'].mean())
print("Recall: ", GBDT_b_scores['test_recall'].mean())

Mean G-mean: 0.306 (+/- 0.092)
F1:  0.08353109931520726
Precision:  0.15558237840590783
FDR:  0.10217391304347825
FAR : 8.860801636387675e-05
Recall:  0.10217391304347825
CPU times: user 141 ms, sys: 76 ms, total: 217 ms
Wall time: 32.2 s


In [35]:
%%time
from imblearn.ensemble import BalancedBaggingClassifier
# cv=5 and all features + BaggingClassifier
BG_pipe =  Pipeline([("impute", SimpleImputer(strategy='constant',fill_value= 0)),("scale", MaxAbsScaler()),("BG", BalancedBaggingClassifier())])
BG_scores = cross_validate(BG_pipe, X, y, cv=5, scoring=scoring, n_jobs=-1)
# print the mean G-mean score and standard deviation
print("Mean G-mean: {:.3f} (+/- {:.3f})".format(BG_scores['test_gmean'].mean(), BG_scores['test_gmean'].std()))
print("F1: ", BG_scores['test_f1'].mean())
print("Precision: ", BG_scores['test_precision'].mean())
print("FDR: ", BG_scores['test_fdr'].mean())
print("FAR :", BG_scores['test_far'].mean())
print("Recall: ", BG_scores['test_recall'].mean())

Mean G-mean: 0.881 (+/- 0.040)
F1:  0.0022668784006381567
Precision:  0.0011350775268555093
FDR:  0.8217391304347826
FAR : 0.05273554011761102
Recall:  0.8217391304347826
CPU times: user 220 ms, sys: 887 ms, total: 1.11 s
Wall time: 2min 59s


In [38]:
%%time
# cv=5 and 5 features + BG
BG_b_pipe =  Pipeline([("impute", SimpleImputer(strategy='constant',fill_value= 0)),("scale", MaxAbsScaler()),("BG", BalancedBaggingClassifier())])
BG_b_scores = cross_validate(BG_b_pipe, X_b, y_b, cv=5, scoring=scoring, n_jobs=-1)
# print the mean G-mean score and standard deviation
print("Mean G-mean: {:.3f} (+/- {:.3f})".format(BG_b_scores['test_gmean'].mean(), BG_b_scores['test_gmean'].std()))
print("F1: ", BG_b_scores['test_f1'].mean())
print("Precision: ", BG_b_scores['test_precision'].mean())
print("FDR: ", BG_b_scores['test_fdr'].mean())
print("FAR :", BG_b_scores['test_far'].mean())
print("Recall: ", BG_b_scores['test_recall'].mean())

Mean G-mean: 0.894 (+/- 0.040)
F1:  0.002634887643297783
Precision:  0.001319531441351816
FDR:  0.8391304347826086
FAR : 0.044615325954591015
Recall:  0.8391304347826086
CPU times: user 146 ms, sys: 94.8 ms, total: 241 ms
Wall time: 4.69 s
